In [16]:
# Corpus from https://www.kaggle.com/datasets/chazzer/big-language-detection-dataset?resource=download

In [45]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.simplefilter("ignore")

Preprocess corpus data in bulk

In [46]:
topLang = ["eng", "cmn", "hin", "spa", "fra", "ara", "ben", "rus", "por", "ind", "urd", "deu", "jpn", "swh", "pnb", "tam", "kor", "vie", "jav", "ita", "tha", "tgl", "pol", "yor", "ukr", "ibo", "npi", "ron", "nld", "zsm", "afr", "grc", "swe", "heb", "lat", "san", "gle", "mri", "chr", "nav", "haw", "smo"]

data = pd.read_csv("sentences.csv")

data.drop(["id"], axis=1, inplace=True)
data = data[data["lan_code"].isin(topLang)]

data["sentence"] = data.apply(lambda x: re.sub(r"(?=[\p{Common}])[^']|(?<![a-zA-Z])'|'(?![a-zA-Z])", " ", x["sentence"].lower()), axis=1)

""" To enforce all languages have an equal number of samples:"""
# data = filtered_data.groupby("lan_code").sample(28)

""" To just ensure that language sample sizes are not larger than 1000:"""
data = data[data.groupby("lan_code").cumcount() < 1000]

data.to_csv("language_corpus.csv", encoding='utf-8', index=False)

Test Model

In [88]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB, MultinomialNB
from sklearn.metrics import accuracy_score

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer

def returnSelf(x):
    return x

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordPieceTrainer(vocab_size=50000, special_tokens=["[UNK]"])
tokenizer.train(["language_corpus.csv"], trainer)

data = pd.read_csv("language_corpus.csv")
tokenized_data = tokenizer.encode_batch(data["sentence"])

tokenized_data = [i.tokens for i in tokenized_data]

le = LabelEncoder()
le.fit(data["lan_code"])
labels = le.transform(data["lan_code"])

train_text, test_text, train_labels, test_labels = train_test_split(tokenized_data, labels, test_size=0.2)

vectorizer = TfidfVectorizer(ngram_range=(1,3), analyzer=returnSelf)
vectorizer.fit(train_text)
train_text = vectorizer.transform(train_text)
test_text = vectorizer.transform(test_text)

languageDetectionModel = ComplementNB()
languageDetectionModel.fit(train_text, train_labels)

predictions = languageDetectionModel.predict(test_text)

ac = accuracy_score(test_labels, predictions)

print("accuracy: " + str(ac))




accuracy: 0.963096168881939
